In [1]:
from __future__ import annotations
import uproot
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import plotly
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots
from typing import overload
from abc import ABC, abstractmethod
import numpy as np
import sympy
from sympy.integrals.transforms import laplace_transform
import random
import math
from tqdm.auto import tqdm, trange
from concurrent.futures import ThreadPoolExecutor
import scipy.stats as stats
from scipy.optimize import curve_fit


from Class1 import parametricValue

    
from Class1 import fixedValue


from Class1 import parametricValueNumpy


from Class1 import parametricValueSympy


from Class1 import binning


from Class1 import discretepdf


from Class1 import NLL


from Class1 import lumiDist


from Class1 import lumiDistSympyLaplace


from Class1 import lumiDistNumpy


from Class1 import SimulationVSPu


In [2]:
from Class1 import RunInfo


In [3]:
from pathlib import Path

import sys
sys.path.append('../nTuplizer')

from allInfo import GetAllRuns, GetAllEras

def export_graph(g , n , pname, pdir):
    ghtml = g.to_html()
    Path('{0}/{1}/'.format(pdir , pname) ).mkdir(parents=True, exist_ok=True)
    with open('{0}/{1}/{2}.html'.format(pdir , pname , n) , 'w') as f:
        f.write(ghtml)
def publish_res(rinfo , gchi2 , vname ,  name , publish_dir):

    
    
    
    gg = rinfo.plot_lumi_distribution(subRuns=-1 , colorLumiScale=4 , density=False)
    gg.update_layout(
        title="Distribution of BX luminosity",
        xaxis_title='luminosity mub/25ns',
        yaxis_title="probability",
        legend_title="Runs",
    )
    export_graph(gg , '01-lumi_distribution' , name , publish_dir)
    
    
    
    
    #02-pu_distribution
    
    gg = rinfo.plotPUDists(70)
    export_graph(gg , '02-pu_distribution' , name , publish_dir)
    
    
    
    
    
    #03-{vname}_distribution:
    
    if rinfo.parentRun is None:
        gg = rinfo.plotDataDist(zoom=True)
        export_graph(gg , '03-{0}_distribution'.format(vname) , name , publish_dir) 

    
    
    
    #04-{vname}_predictions:

    gg = rinfo.plotPredictions([60,65,75,80,90])
    export_graph(gg , '04-{0}_predictions'.format(vname) , name , publish_dir)
    
        
        
        
        
    for _xsec in [60,65,75,80,90]:
        gg = rinfo.plotRunPredictions(_xsec)
        export_graph(gg , '05-{0}_predictions_xsec{1}'.format(vname , _xsec) , name , publish_dir)
            
    
    
    export_graph(gchi2 , '06-bestfit_details'  , name , publish_dir)
    export_graph(rinfo.postFitPlots(), '07-postFit_plots'  , name , publish_dir)
    export_graph(rinfo.pullPlots(50 , 1.3) , '08-pullPlots' , name , publish_dir)
    export_graph(rinfo.NadjiehPullPlots(50 , 1.3) , '08-NadjiehPullPlots' , name , publish_dir)
    
    
    
    
    
    if rinfo.parentRun is None:
        export_graph(rinfo.aggregateFitRes() , '09-summary1' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes2() , '09-summary2' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes3() , '09-summary3' , name ,publish_dir)
        export_graph(rinfo.aggregateFitRes4() , '09-summary4' , name ,publish_dir)
        
        
   
rinfo = None
        
def process_and_publish(year , era , vname , vmin , vmax , vnbins , pu_max ,
                        lumiName = 'PHYSICSDel' , lumiQ =  np.array([0,0.05, 0.1,0.5,1.0]) , lnbins_per_q = 10 ,
                        xsec = np.arange(40,100,1), nthreads = 30 , publish_dir = 
                        '/eos/user/c/cmstandi/www/PU/newres/NimaTest6/1' ):
    nbins = lumiQ.size -1
  #  print(nbins)    


    name = '{0}{1}/{2}'.format(era, year , vname)
    var_bins = np.linspace(vmin,vmax,vnbins+1)
    simDist = SimulationVSPu(vname , year = year , var_bins=var_bins , pu_max=pu_max , nthreads=nthreads)


    
    g = simDist.plot(param=1)
    simDist.plot(param=10 , g=g)
    simDist.plot(param=20 , g=g)
    simDist.plot(param=30 , g=g)
    simDist.plot(param=40 , g=g)
    simDist.plot(param=50 , g=g)
    simDist.plot(param=60 , g=g)
    simDist.plot(param=70 , g=g)
    simDist.plot(param=80 , g=g)
    simDist.plot(param=90 , g=g)
    g.update_layout(
        title="Simulated distribution for PU",
        xaxis_title='Pile Up',
        yaxis_title="probability",
        legend_title="param values",
    )    
    export_graph(g , '00-Simulation' , name , publish_dir )
    
    
    
    
    allRuns = sorted( list(set( GetAllRuns(year , era) ) ) )
    allRuns = allRuns[3:6]
    #print(allRuns)
    print('is going to run over {0} runs'.format( len(allRuns)) )
    print('Number of Lumi Bins = {0}'.format(nbins))

    
    rinfo = RunInfo(0 , vname , var_bins , lumiName ,lumiQ , nbins_perq=lnbins_per_q , sub_runs=allRuns , 
                    nthreads=nthreads , xsecs = xsec )
    
    rinfo.setSimulation(simDist)
    gchi2 = rinfo.fit()
    
#     print(vname)
#     print(var_bins)
#     print(lumiName)
#     print(lumiQ)
#     print(lnbins_per_q)
    
    try:
        publish_res(rinfo , gchi2 , vname , name , publish_dir)
        for sr in rinfo._subRuns : #+ rinfo._subRunsSameLumiBins:
            #ext = '/SRSimilarBinning' if sr._isSecondHand else '/SRSameBinning'
            publish_res(sr , None , vname , vname , '{0}/{1}{2}/Runs/Run{3}'.format(publish_dir , era, year , sr.run) )
    except Exception as e:
        print(e)
        return rinfo
    #del rinfo
    #del simDist
    return rinfo


In [4]:
variables = { "nVertices" : ( "nVertices" , 90 , 0 , 90 ) ,
              "nGoodVertices" : ("nGoodVertices", 80, 0 , 80) ,
              "nEles" : ("nEles" , 10 , 0 , 10 ) ,
              "nMus" : ("nMus" , 10 , 0 , 10 ),
              "nChargedHadrons" : ("nChargedHadrons" , 120 , 0 , 1200 ),
              "nLostTracks": ("nLostTracks" , 35 , 0 , 35 ),
              "nPhotons" : ("nPhotons" , 120 , 0 , 600 ),
              "nNeutralHadrons" : ("nNeutralHadrons" , 60 , 0 , 120 ),
              "fixedGridRhoAll" : ("fixedGridRhoAll" , 40 , 0 , 40 ),
              "fixedGridRhoFastjetAll" : ("fixedGridRhoFastjetAll" , 40 , 0 , 40 ),
              "fixedGridRhoFastjetAllCalo" : ("fixedGridRhoFastjetAllCalo" , 25 , 0 , 25 ),
              "fixedGridRhoFastjetCentral" : ("fixedGridRhoFastjetCentral" , 50 , 0 , 50 ),
              "fixedGridRhoFastjetCentralCalo" : ("fixedGridRhoFastjetCentralCalo" , 20 , 0 , 20 ),
              "fixedGridRhoFastjetCentralChargedPileUp" : ("fixedGridRhoFastjetCentralChargedPileUp" , 35 , 0 , 35 ),
              "fixedGridRhoFastjetCentralNeutral" : ("fixedGridRhoFastjetCentralNeutral" , 12 , 0 , 12 )
            }

vvv = {}
for v,vinfo in variables.items():
     vvv[v] = process_and_publish(2018 , 'B' , vinfo[0] , vinfo[2] , vinfo[3] , vinfo[1] , 90  )


  0%|          | 0/92 [00:00<?, ?it/s, SIMULATION]

is going to run over 3 runs
Number of Lumi Bins = 4


  0%|          | 0/17 [00:00<?, ?it/s, RUN 0]

  0%|          | 0/11 [00:00<?, ?it/s, RUN 317212]

  0%|          | 0/11 [00:00<?, ?it/s, RUN 317182]

...

  0%|          | 0/4 [00:00<?, ?it/s, RUN 317212]

  0%|          | 0/4 [00:00<?, ?it/s, RUN 317182]

number of Runs = 3
Number of Lumi Bins = 4


  0%|          | 0/7 [00:00<?, ?it/s, RUN 0]

In order to reference traces by row and column, you must first use plotly.tools.make_subplots to create the figure with a subplot grid.


In [5]:
#  vvv['nVertices']._subRunsSameLumiBins[0].fitResults

In [6]:
# vvv['nVertices'].fitResults[3].plot()

In [7]:
# vvv['nVertices'].predictions[0] 

In [8]:
# vvv['nVertices'].data_hists[0]

In [9]:
# vvv['nVertices'].fitResults

In [10]:
for key in vvv:
    for i in range(vvv[key].nLumiBins):
        print('best Cross Section fit for var = {0} in lumi bin = {1} is: '.format(vvv[key].vname,i+1),  vvv[key].fitResults[i].bestFit)

best Cross Section fit for var = nVertices in lumi bin = 1 is:  76.0
best Cross Section fit for var = nVertices in lumi bin = 2 is:  72.0
best Cross Section fit for var = nVertices in lumi bin = 3 is:  73.0
best Cross Section fit for var = nVertices in lumi bin = 4 is:  75.0
best Cross Section fit for var = nGoodVertices in lumi bin = 1 is:  66.0
best Cross Section fit for var = nGoodVertices in lumi bin = 2 is:  64.0
best Cross Section fit for var = nGoodVertices in lumi bin = 3 is:  64.0
best Cross Section fit for var = nGoodVertices in lumi bin = 4 is:  66.0
best Cross Section fit for var = nEles in lumi bin = 1 is:  79.0
best Cross Section fit for var = nEles in lumi bin = 2 is:  79.0
best Cross Section fit for var = nEles in lumi bin = 3 is:  79.0
best Cross Section fit for var = nEles in lumi bin = 4 is:  79.0
best Cross Section fit for var = nMus in lumi bin = 1 is:  64.0
best Cross Section fit for var = nMus in lumi bin = 2 is:  63.0
best Cross Section fit for var = nMus in lum